In [ ]:
!pip install openai
!pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [36]:
import os
import pandas as pd
from openai import OpenAI
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "Enter_API_key"

# Initialize the OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Load the Excel file containing test cases
df = pd.read_excel('/content/drive/MyDrive/sample_test_cases.xlsx')

# Preprocess the data (convert Yes/No to 1/0 for machine learning)
df['Test Case Passed (Yes/No)'] = df['Test Case Passed (Yes/No)'].map({'Yes': 1, 'No': 0})
df['Failure Reason'] = df['Failure Reason'].fillna('No Failure')

# Feature engineering (e.g., failure reason length)
df['Failure Length'] = df['Failure Reason'].apply(len)

# Group by test case ID (assuming there's a 'Test Case ID' column)
grouped_df = df.groupby('Test Case ID')

# Initialize a new DataFrame to hold the final predictions and AI suggestions
final_predictions = []

# Function to predict whether a test case will pass or fail overall
def predict_test_case_outcome(test_case_data):
    # Prepare data for machine learning based on the aggregated data of this test case
    X = test_case_data[['Failure Length']]
    y = test_case_data['Test Case Passed (Yes/No)']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a RandomForest model (or any other model)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Make a prediction based on the entire test case's data
    return model.predict(X_test).mean() >= 0.5  # Predict 1 (pass) if the average prediction is >= 0.5

# Function to get fix suggestions using OpenAI
def get_fix_suggestions(failure_reason):
    if failure_reason == 'No Failure':  # No suggestion for successful cases
        return 'N/A'
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"The test case failed due to the following reason: {failure_reason}. How can this issue be fixed?"}
            ]
        )
        # Return the suggested fix
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Process each test case group
for test_case_id, test_case_data in grouped_df:
    # Predict the overall outcome for this test case
    will_pass = predict_test_case_outcome(test_case_data)

    # If the test case will fail, generate AI suggestions
    failure_reasons = test_case_data['Failure Reason'].unique()  # Get unique failure reasons
    ai_suggestions = []

    if not will_pass:  # Only generate suggestions for failing cases
        for reason in failure_reasons:
            if reason != 'No Failure':
                suggestion = get_fix_suggestions(reason)
                ai_suggestions.append(f"{reason}: {suggestion}")

    # Summarize the results into a new row for the final predictions DataFrame
    final_row = {
        'Test Case ID': test_case_id,
        'Overall Prediction (Pass/Fail)': 'Yes' if will_pass else 'No',
        'AI Suggestions': "; ".join(ai_suggestions) if ai_suggestions else 'N/A'
    }

    final_predictions.append(final_row)

# Convert final predictions list to DataFrame
final_predictions_df = pd.DataFrame(final_predictions)

# Save the final predictions and AI suggestions to a new Excel file
output_file = 'test_case_overall_predictions_with_ai_suggestions.xlsx'
final_predictions_df.to_excel(output_file, index=False)

print(f"Overall predictions with AI suggestions saved to {output_file}")

# (Optional) If you're using Google Colab, download the output file
from google.colab import files
files.download(output_file)


Overall predictions with AI suggestions saved to test_case_overall_predictions_with_ai_suggestions.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>